In [1]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold, train_test_split
# from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, precision_recall_curve, average_precision_score

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# from link.src.py_scripts.process_pals import begin_processing

In [2]:
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8

In [3]:
random_seed = 17

In [4]:
SEQUENCE_BASED = "sequence_based"
PHYS_CHEM_STRUCT = "Phys-Chem-Struct"
LAST_50_BP = "50b.p._Stats"

# Data preparation

In [85]:
def get_trinucleotides(lst):
    """
    '1234' -> ['123','234']
    """
    res = []
    for i in range(len(lst) - 2):
        res.append(lst[i] + lst[i + 1] + lst[i + 2])
    return res


def get_dinucleotides(lst):
    """
    '123' -> ['12', '23']
    """
    res = []
    for i in range(len(lst) - 1):
        res.append(lst[i] + lst[i + 1])
    return res


def annotate_with_di_tri_nucleotides(lines):
    annotated_rows = []
    for line in lines:
        row = {}
        for di_n in get_dinucleotides(line) + get_trinucleotides(line):
            row[di_n] = row.get(di_n, 0) + 1
        annotated_rows.append(row)
    return pd.DataFrame(annotated_rows).fillna(0)


def train_model(true_paths, false_paths, model_type):
    if model_type == LAST_50_BP:
        true_sequences = []
        for path in true_paths:
            # True positive data
            with open(path, 'r') as f:
                true_sequences += list(map(lambda x: x.strip(), f.readlines()))

        false_sequences = []
        for path in false_paths:
            # True positive data
            with open(path, 'r') as f:
                false_sequences += list(map(lambda x: x.strip(), f.readlines()))

        df_True = annotate_with_di_tri_nucleotides(np.unique(true_sequences))
        df_False = annotate_with_di_tri_nucleotides(np.unique(false_sequences))
    else:
        df_True = pd.concat(list(map(lambda x: pd.read_csv(x, index_col=0, sep=';'), true_paths))).drop_duplicates()
        df_False = pd.concat(list(map(lambda x: pd.read_csv(x, index_col=0, sep=';'), false_paths))).drop_duplicates()

    if df_True.shape[0] > df_False.shape[0]:
        df_True_n = df_True.sample(df_False.shape[0], random_state=random_seed)
        df_False_n = df_False
    else:
        df_True_n = df_True
        df_False_n = df_False.sample(df_True.shape[0], random_state=random_seed)

    X = pd.concat([df_True_n, df_False_n], ignore_index=True)
    Y = pd.Series(np.append(np.full(df_True_n.shape[0], 1), np.full(df_False_n.shape[0], 0)))

    if model_type == PHYS_CHEM_STRUCT:
        # Getting rid of unneeded columns
        X = X[X.columns[np.array(X.columns.map(
            lambda x: (len(x) > 4 or 'LP' in x or 'LB' in x or 'RB' in x) and 'GC' not in x),
            dtype='bool')]]
    elif model_type == SEQUENCE_BASED:
        # Getting rid of unneeded columns
        X = X[X.columns[(X.columns.map(len) < 4)]]

    rf = RandomForestClassifier(n_jobs=-1, n_estimators=2000)

    return rf.fit(X, Y)


def evaluate(model, model_type, trained_on, eval_path, evaluated_on, write=True):
    if model_type == LAST_50_BP:
        eval_50_bp = []
        with open(eval_path, 'r') as file:
            for line in file:
                eval_50_bp.append(line.strip())
        df_Eval = annotate_with_di_tri_nucleotides(eval_50_bp)
    elif model_type == PHYS_CHEM_STRUCT:
        df_Eval = pd.read_csv(eval_path, index_col=0, sep=';')

        df_Eval = df_Eval[df_Eval.columns[np.array(
            df_Eval.columns.map(
                lambda x: (len(x) > 4 or 'LP' in x or 'LB' in x or 'RB' in x) and 'GC' not in x
            ),
            dtype='bool'
        )]]
    elif model_type == SEQUENCE_BASED:
        df_Eval = pd.read_csv(eval_path, index_col=0, sep=';').drop_duplicates()
        df_Eval = df_Eval[df_Eval.columns[(df_Eval.columns.map(len) < 4)]]
    else:
        raise ValueError('Wrong model type')

    Y_pred_Eval = model.predict(df_Eval)
    evaluation_result = np.mean(Y_pred_Eval)
    if write:
        with open(f'Extra_recognition.csv', 'a') as file:
            file.write(f'{model_type},{trained_on},{evaluated_on},{evaluation_result}\n')
    return evaluation_result


In [86]:
L1_50_BP_PATH = 'L1_50_last.txt'
L1_50_BP_SHUFFLED_PATH = 'L1_50_last_shuffled.txt'

ALU_50_BP_PATH = '../../data/sequences/Alu_All_50_bs_2.txt'
ALU_50_BP_SHUFFLED_PATH= 'alu_50_last_shuffled.csv'

# ALU_S_PATH = 'AluS3UTR.csv'
# ALU_Y_PATH = 'AluY3UTR.csv'
ALU_PATH = 'Alu3UTR.csv'
ALU_SHUFFLED_PATH = 'Shuffled_Alu.csv'

L1_PATH = 'L13UTR.csv'
L1_SHUFFLED_PATH = 'Shuffled_L1.csv'

PSEUDOGENES_50_LAST_PATH = 'pseudogenes_50_last.txt'
MRNA_50_LAST_PATH = 'KnownGene_50_last.txt'

MRNA_PATH = 'KnownGene3End.csv'
PSEUDOGENES_PATH = 'Pseudogenes3End.csv'

RP_PATH = 'RP3End.csv'
RP_50_LAST_PATH = 'RP_50_last_shuffled.csv'


## Models

### Alu, L1

In [87]:
alu_l1_vs_shuffle_seq_based = train_model(
    [  # True positive data
        ALU_PATH,
        L1_PATH,
    ],
    [  # True negative data
        ALU_SHUFFLED_PATH,
        L1_SHUFFLED_PATH,
    ],
    SEQUENCE_BASED,  # Model type
)

alu_l1_vs_shuffle_phys_chem = train_model(
    [   # True positive data
        ALU_PATH,
        L1_PATH,
    ],
    [  # True negative data
        ALU_SHUFFLED_PATH,
        L1_SHUFFLED_PATH,
    ],
    PHYS_CHEM_STRUCT, # Model type
)

alu_l1_vs_shuffle_50_bp = train_model(
    [  # True positive data
        ALU_50_BP_PATH,
        L1_50_BP_PATH,
    ],
    [  # True negative data
        L1_50_BP_SHUFFLED_PATH,
        ALU_50_BP_SHUFFLED_PATH,
    ],
    LAST_50_BP,  # Model type
)


### Alu

In [88]:
alu_vs_shuffle_seq_based = train_model(
    [  # True positive data
        ALU_PATH,
    ],
    [  # True negative data
        ALU_SHUFFLED_PATH,
    ],
    SEQUENCE_BASED,  # Model type
)

alu_vs_shuffle_phys_chem = train_model(
    [   # True positive data
        ALU_PATH,
    ],
    [  # True negative data
        ALU_SHUFFLED_PATH,
    ],
    PHYS_CHEM_STRUCT, # Model type
)

alu_vs_shuffle_50_bp = train_model(
    [  # True positive data
        ALU_50_BP_PATH,
    ],
    [  # True negative data
        ALU_50_BP_SHUFFLED_PATH,
    ],
    LAST_50_BP,  # Model type
)

### L1

In [89]:
l1_vs_shuffle_seq_based = train_model(
    [  # True positive data
        L1_PATH,
    ],
    [  # True negative data
        L1_SHUFFLED_PATH,
    ],
    SEQUENCE_BASED,  # Model type
)

l1_vs_shuffle_phys_chem = train_model(
    [   # True positive data
        L1_PATH,
    ],
    [  # True negative data
        L1_SHUFFLED_PATH,
    ],
    PHYS_CHEM_STRUCT, # Model type
)

l1_vs_shuffle_50_bp = train_model(
    [  # True positive data
        L1_50_BP_PATH,
    ],
    [  # True negative data
        L1_50_BP_SHUFFLED_PATH,
    ],
    LAST_50_BP,  # Model type
)

## Evaluations

In [90]:
with open(f'Extra_recognition.csv', 'w') as file:
    file.write(f'Model type,Train classes,Recognition class,% recognized\n')

### Alu, L1 ==> p_pseudo

In [91]:
print(evaluate(
    alu_l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_L1_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_L1_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_L1_vs_Shuffled',
    PSEUDOGENES_50_LAST_PATH,
    'P_Pseudo',
    write=True
))


0.292239010989011
0.16672169032684053
0.16503866503866504


### Alu, L1 ==> mRNA

In [92]:
print(evaluate(
    alu_l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_L1_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_L1_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_L1_vs_Shuffled',
    MRNA_50_LAST_PATH,
    'mRNA',
    write=True
))


0.48789454509323354
0.3588019203483061
0.40915491477690674


### Alu ==> p_pseudo

In [93]:
print(evaluate(
    alu_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    alu_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    alu_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_vs_Shuffled',
    PSEUDOGENES_50_LAST_PATH,
    'P_Pseudo',
    write=True
))


0.29086538461538464
0.11026741498844503
0.10236060236060236


### Alu ==> mRNA

In [94]:
print(evaluate(
    alu_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    alu_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    alu_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_vs_Shuffled',
    MRNA_50_LAST_PATH,
    'mRNA',
    write=True
))


0.5638393005001159
0.2894905634706214
0.19528803514007692


### Alu ==> RP

In [95]:
print(evaluate(
    alu_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    alu_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    alu_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_vs_Shuffled',
    RP_50_LAST_PATH,
    'RP',
    write=True
))

0.6326530612244898
0.29842931937172773
0.17333333333333334


### Alu ==> L1

In [96]:
print(evaluate(
    alu_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_vs_Shuffled',
    L1_PATH,
    'L1',
    write=True
))
print(evaluate(
    alu_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_vs_Shuffled',
    L1_PATH,
    'L1',
    write=True
))
print(evaluate(
    alu_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_vs_Shuffled',
    L1_50_BP_PATH,
    'L1',
    write=True
))

0.2005420054200542
0.023114754098360654
0.017517366354575657


### L1 ==> Alu

In [97]:
print(evaluate(
    l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'L1_vs_Shuffled',
    ALU_PATH,
    'Alu',
    write=True
))
print(evaluate(
    l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'L1_vs_Shuffled',
    ALU_PATH,
    'Alu',
    write=True
))
print(evaluate(
    l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'L1_vs_Shuffled',
    ALU_50_BP_PATH,
    'Alu',
    write=True
))


0.581918505942275
0.06975621530292059
0.013034033309196235


### L1 ==> p_pseudo

In [98]:
print(evaluate(
    l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'L1_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'L1_vs_Shuffled',
    PSEUDOGENES_PATH,
    'P_Pseudo',
    write=True
))
print(evaluate(
    l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'L1_vs_Shuffled',
    PSEUDOGENES_50_LAST_PATH,
    'P_Pseudo',
    write=True
))

0.40041208791208793
0.1772862330802245
0.32336182336182334


### L1 ==> mRNA

In [99]:
print(evaluate(
    l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'L1_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'L1_vs_Shuffled',
    MRNA_PATH,
    'mRNA',
    write=True
))
print(evaluate(
    l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'L1_vs_Shuffled',
    MRNA_50_LAST_PATH,
    'mRNA',
    write=True
))

0.41284403669724773
0.26486423448463525
0.33780290452071204


### L1 ==> RP

In [100]:
print(evaluate(
    l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'L1_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'L1_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'L1_vs_Shuffled',
    RP_50_LAST_PATH,
    'RP',
    write=True
))


0.37755102040816324
0.2513089005235602
0.2866666666666667


### Alu, L1 ==> RP

In [101]:
print(evaluate(
    alu_l1_vs_shuffle_seq_based,
    SEQUENCE_BASED,
    'Alu_L1_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_phys_chem,
    PHYS_CHEM_STRUCT,
    'Alu_L1_vs_Shuffled',
    RP_PATH,
    'RP',
    write=True
))
print(evaluate(
    alu_l1_vs_shuffle_50_bp,
    LAST_50_BP,
    'Alu_L1_vs_Shuffled',
    RP_50_LAST_PATH,
    'RP',
    write=True
))


0.4897959183673469
0.3717277486910995
0.41333333333333333


### Alu, L1 ==> Alu

In [102]:
# print(evaluate(
#     alu_l1_vs_shuffle_seq_based,
#     SEQUENCE_BASED,
#     'Alu_L1_vs_Shuffled',
#     ALU_PATH,
#     'Alu',
#     write=True
# ))
# print(evaluate(
#     alu_l1_vs_shuffle_phys_chem,
#     PHYS_CHEM_STRUCT,
#     'Alu_L1_vs_Shuffled',
#     ALU_PATH,
#     'Alu',
#     write=True
# ))
# print(evaluate(
#     alu_l1_vs_shuffle_50_bp,
#     LAST_50_BP,
#     'Alu_L1_vs_Shuffled',
#     ALU_50_BP_PATH,
#     'Alu',
#     write=True
# ))


### Alu, L1 ==> L1

In [103]:
# print(evaluate(
#     alu_l1_vs_shuffle_seq_based,
#     SEQUENCE_BASED,
#     'Alu_L1_vs_Shuffled',
#     L1_PATH,
#     'L1',
#     write=True
# ))
# print(evaluate(
#     alu_l1_vs_shuffle_phys_chem,
#     PHYS_CHEM_STRUCT,
#     'Alu_L1_vs_Shuffled',
#     L1_PATH,
#     'L1',
#     write=True
# ))
# print(evaluate(
#     alu_l1_vs_shuffle_50_bp,
#     LAST_50_BP,
#     'Alu_L1_vs_Shuffled',
#     L1_50_BP_PATH,
#     'L1',
#     write=True
# ))
